In [ ]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%reload_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import sys
sys.path.append('../src')



import cv2 as cv
from cellpose import models, io, plot

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle
import re
import tifffile as tiff
from time import time
from tqdm import tqdm
from utils import list_subdir_filter as lsd#, slide_cycle_dot as scd
from metadata import metadata

md = metadata()

In [ ]:
def unique(list_):
    return sorted(list(set(list_)))

def get_id(f):
    return re.sub('^.*A40\.([0-9]+).*$', '\\1', f)


all_samples = lsd(os.path.join(md.folders['images'], 'mouse'))
all_img_files = lsd(md.folders['bg_removed'])
all_seg_files = lsd(md.folders['segmented'], True, 'whole')

all_sample_ids, all_img_ids = list(map(
    lambda x: unique([get_id(f) for f in x]), 
    [all_samples, all_img_files]
))

all_seg_ids = unique([re.sub('^.*segmentation_([0-9]+).*$', '\\1', f) for f in all_seg_files])

out_dir = os.path.join(md.folders['segmented'], 'cellpose_new')
Path(out_dir).mkdir(exist_ok=True, parents=True)

model = models.Cellpose(gpu=True, model_type='nuclei')

In [ ]:
all_seg_ids

In [ ]:
to_segment = all_img_files#[f for f in all_img_files if re.search('(2294|2407)', f)]#re.sub('^.*A40\.([0-9]+).*$', '\\1', f) not in all_seg_ids]
to_segment

In [ ]:
39618 / 5

In [ ]:
%%time
for img_file in to_segment:
    
    tissue_id =re.sub('.*A40\.([0-9]{4}).*$', '\\1', img_file)
    print(tissue_id)
    
    img = np.load(img_file)[0]
    max_size = np.shape(img)[0]
    width = 8000
    
    for i in tqdm(range((max_size // width) + 1)):
        print(i)
        min_row, max_row = max(0, i * width - 100), min(max_size, (i+1) * width + 100)
        masks, flows, styles, diams = model.eval(img[min_row:max_row], diameter=None, channels=[0,0])
        pickle_dict = {'masks':masks, 'flows':flows, 'styles':styles, 'diams':diams}
        
        np.save(f'T3_A40_{tissue_id}_dapi1_masks_{i:02}.npy', masks)
        
        with open(os.path.join(out_dir, f'TT_A40_{tissue_id}_dapi1_{i:02}.pkl'), 'wb') as handle:
            pickle.dump(pickle_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%time
masks, flows, styles, diams = model.eval(ii[:, :3300], diameter=None, channels=[0,0])